# 题目

>给你一个整数数组 nums ，数组中共有 n 个整数。132 模式的子序列 由三个整数 nums[i]、nums[j] 和 nums[k] 组成，并同时满足：i < j < k 和 nums[i] < nums[k] < nums[j] 。  
如果 nums 中存在 132 模式的子序列 ，返回 true ；否则，返回 false 。

# 方法一：枚举1

> 考虑枚举其中1个的下标，并用栈维护另外两个下标的可能值。  
从右往左枚举'1'，并使用一个自底向上递减的单调栈保存'2'的候选元素，具体过程如下：  
1. 创建一个栈，其中的初始元素为数组的尾部元素；
2. 从右边第二个元素开始向左遍历，每次遇到新元素时进行以下判断：  
a. 判断其能否作为'1'；
b. 判断其能否作为'3'；
c. 判断其能否作为'2'的候选。

## 复杂度

- 时间复杂度: $O(n)$ ，其中 $n$ 是数组的长度。

> 每一个元素最多被加入和弹出单调栈各一次。

- 空间复杂度: $O(n)$ ，其中 $n$ 是数组的长度。

> 单调栈需要使用的空间。

## 代码

In [1]:
def find132pattern(nums):
    n = len(nums)
    candidate_k = [nums[n - 1]]  # 用于保存'2'的候选元素的单调栈（自底向上递减），从右边第2个元素开始
    max_k = float("-inf")  # 记录能够作为'2'的数的最大值（值越大，'1'越容易小于'2'），初始化为-∞

    for i in range(n - 2, -1, -1):  # 从右向左遍历
        # 1、判断当前数是否可以作为'1'
        if nums[i] < max_k:
            return True  # 若是，返回True
        # 2、判断当前数是否可以作为'3'
        # 若当前数大于栈顶数，就不断弹出栈顶元素直至当前数小于栈顶数或栈为空，记录弹出数中最大的一个（目前为止最适合作为'2'的数）
        while candidate_k and nums[i] > candidate_k[-1]:
            max_k = candidate_k[-1]
            candidate_k.pop()
        # 3、若当前数比之前最适合作为'2'候选的数max_k更大，将当前数作为'2'的候选加入栈
        if nums[i] > max_k:
            candidate_k.append(nums[i])

    return False

#### 测试一

In [2]:
nums = [1,2,3,4]
find132pattern(nums)

False

#### 测试二

In [3]:
nums = [3,1,4,2]
find132pattern(nums)

True

# 方法二：枚举2

> 当我们枚举 2 的下标 k 时，与方法一相反，从左到右进行枚举的方法是十分合理的：在枚举的过程中，i,j 的下标范围都是增加的。  
由于我们需要保证 1<2 并且 2<3 ，那么我们需要维护一系列尽可能小的元素作为 1 的候选元素，并且维护一系列尽可能大的元素作为 3 的候选元素。  
我们可以分情况进行讨论，假设当前有一个小元素 $x_i$ 和一个大元素 $x_j$ 表示一个二元组，而我们当前遍历到了一个新的元素 x=a[k] ，那么：  
1. 如果 x>$x_j$ ，那么让 x 作为 3 显然是比 $x_j$ 作为 3 更优，因此我们可以用 x 替代 $x_j$ ；
2. 如果 x<$x_i$ ，那么让 x 作为 1 显然是比 $x_i$ 作为 3 更优，然而我们必须要满足 132 模式中的顺序，即 1 出现在 3 之前，这里如果我们简单地用 x 替代 $x_i$ ，那么 $x_i$=x 作为 1 是出现在 $x_j$ 作为 3 之后的，这并不满足要求。因此我们需要为 x 找一个新的元素作为 3 。由于我们还没有遍历到后面的元素，因此可以简单地将 x 同时看作一个二元组的 $x_i$ 和 $x_j$ ；
4. 对于其它的情况，$x_i$≤x≤$x_j$ ，x 无论作为 1 还是 3 都没有当前二元组对应的要优，因此我们可以不用考虑 x 作为 1 或者 3 的情况。

> 这样一来，与方法一类似，我们使用两个单调递减的单调栈维护一系列二元组 $(x_i,x_j)$ ，表示一个可以选择的 1−3 区间，并且从栈底到栈顶 $x_i$ 和 $x_j$ 分别严格单调递减，因为根据上面的讨论，我们只有在 x<$x_i$ 时才会增加一个新的二元组。  
然而与方法一不同的是，如果我们想让 x 作为 2 ，那么我们并不知道到底应该选择单调栈中的哪个 1−3 区间，因此我们只能根据单调性进行二分查找：  
1. 对于单调栈中的 $x_i$ ，需要找出第一个满足 $x_i$<x 的位置 id$x_i$ ，这样从该位置到栈顶的所有二元组都满足 $x_i$<x ；
2. 对于单调栈中的 $x_j$ ，需要找出第一个满足 $x_j$>x 的位置 id$x_j$ ，这样从栈底到该位置的所有二元组都满足 $x_j$>x ；
3. 如果 id$x_i$ 和 id$x_j$ 都存在，且 id$x_i$<id$x_j$ ,那么至少存在一个二元组 $(x_i,x_j)$ 满足 $x_i$<x<$x_j$ ， x 可以作为 2 ，找到了一组满足 132 模式的三元组。  

> 在枚举完所有的元素后，如果仍未找到满足 132 模式的三元组，那就说明其不存在。  
需要注意的是，我们是在单调递减的栈上进行二分查找，因此大部分语言都需要实现一个自定义比较函数，或者将栈中的元素取相反数后再使用默认的比较函数。  

> **这种方法时间复杂度更高，但由于是从左到右遍历，因此不需要提前知道全部数组。**

## 复杂度

- 时间复杂度: $O(nlogn)$ ，其中 $n$ 是数组的长度。

> 二分查找的单次时间为 $O(log⁡n)$ ，一共为 $O(nlog⁡n)$ 。

- 空间复杂度: $O(n)$ ，其中 $n$ 是数组的长度。

> 单调栈需要使用的空间。

## 代码

In [4]:
import bisect

In [5]:
def find132pattern(nums):
    candidate_i, candidate_j = [-nums[0]], [-nums[0]]

    for v in nums[1:]:
        idx_i = bisect.bisect_right(candidate_i, -v)  # bisect_right表示使用二分查找返回-v在列表中插入的位置，靠右侧
        idx_j = bisect.bisect_left(candidate_j, -v)  # bisect_left表示使用二分查找返回-v在列表中插入的位置，靠左侧
        if idx_i < idx_j:
            return True

        if v < -candidate_i[-1]:
            candidate_i.append(-v)
            candidate_j.append(-v)
        elif v > -candidate_j[-1]:
            last_i = -candidate_i[-1]
            while candidate_j and v > -candidate_j[-1]:
                candidate_i.pop()
                candidate_j.pop()
            candidate_i.append(-last_i)
            candidate_j.append(-v)

    return False

#### 测试一

In [6]:
nums = [1,2,3,4]
find132pattern(nums)

False

#### 测试二

In [7]:
nums = [3,1,4,2]
find132pattern(nums)

True